In [16]:
from dataloader.FireSpreadDataset import FireSpreadDataset

dataset_with_doy = FireSpreadDataset(
    data_dir="/Users/sameeps/Documents/Dr-Malof/WIldfire/WildfireSpreadTS/Data/WIldfireSpreadTS_HDF5",
    included_fire_years=[2019],
    n_leading_observations=5,
    crop_side_length=128,
    load_from_hdf5=True,
    is_train=False,
    remove_duplicate_features=False,
    stats_years=(2018, 2020),
    return_doy=True  # ✅ get day-of-year
)


In [17]:
from dataloader.FireSpreadDataset import FireSpreadDataset

dataset_without_doy = FireSpreadDataset(
    data_dir="/Users/sameeps/Documents/Dr-Malof/WIldfire/WildfireSpreadTS/Data/WIldfireSpreadTS_HDF5",
    included_fire_years=[2019],
    n_leading_observations=5,
    crop_side_length=128,
    load_from_hdf5=True,
    is_train=False,
    remove_duplicate_features=False,
    stats_years=(2018, 2020),
    return_doy=False # ✅ get day-of-year
)


In [18]:
# Dataset with DOY
x1, y1, doys1 = dataset_with_doy[0]
print("Sample with DOY:", doys1)

# Dataset without DOY just gives x,y
x2, y2 = dataset_without_doy[0]
print("Shape without DOY:", x2.shape)


Sample with DOY: tensor([66., 67., 68., 69., 70.])
Shape without DOY: torch.Size([5, 40, 128, 128])


In [15]:
from models.SMPTempModel import SMPTempModel

model_with_doy = SMPTempModel(
    encoder_name="resnet18",
    n_channels=x1.shape[1], 
    flatten_temporal_dimension=False,
    pos_class_weight=1.0,
    encoder_weights="imagenet",
    loss_function="Dice",  # ✅ required!
    use_doy=True
)


model_no_doy = SMPTempModel(
    encoder_name="resnet18",
    n_channels=x1.shape[1],
    flatten_temporal_dimension=False,
    pos_class_weight=1.0,
    encoder_weights="imagenet",
    loss_function="Dice",  # ✅
    use_doy=False
)


# (B, T, C, H, W) inputs
x1_batched = x1.unsqueeze(0)
doys1_batched = doys1.unsqueeze(0)

# Run through both models
out_with_doy = model_with_doy(x1_batched, doys1_batched)
out_no_doy = model_no_doy(x1_batched, doys1_batched)

diff = torch.mean(torch.abs(out_with_doy - out_no_doy)).item()
print(f"🚀 Mean absolute diff between DOY vs no DOY: {diff:.6f}")


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /Users/sameeps/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:04<00:00, 11.1MB/s]


Loaded resnet18 with imagenet weights + LTAE
Loaded resnet18 with imagenet weights + LTAE
🚀 Mean absolute diff between DOY vs no DOY: 1.067734
